# Experiment 4
## Aim :- Implementing a  *content-based* recommendation system



Downlaoding a dataset of netflix shows from kaggle.


In [8]:
!kaggle datasets download shivamb/netflix-shows


Dataset URL: https://www.kaggle.com/datasets/shivamb/netflix-shows
License(s): CC0-1.0
  0% 0.00/1.34M [00:00<?, ?B/s]
100% 1.34M/1.34M [00:00<00:00, 136MB/s]


As the dataset gets downlaoded in .zip fomat the following library is used to unarchive it .

In [9]:
!pip install pyunpack patool --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 2.8 MB/s eta 0:00:00


Edit distance is used for the autocorrecting any typos in the movies, and following is the library to do so.

In [49]:
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.7 MB/s eta 0:00:00


Using the pyunpack library for unarchinving the .zip file in /content

In [11]:
from pyunpack import Archive

Archive('/content/netflix-shows.zip').extractall('/content/')

Importing important libraries

In [52]:
import pandas as pd
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity , linear_kernel
import Levenshtein as edit_distance

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Reading the csv file as a pandas dataframe

In [12]:
data = pd.read_csv("/content/netflix_titles.csv")

In [13]:
data.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [14]:
data.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

In [16]:
data.isnull().sum()

,0
show_id,0
type,0
title,0
director,2634
cast,825
country,831
date_added,10
release_year,0
rating,4
duration,3


Making the name of the movies and shows as indices

In [42]:
indices = pd.Series(data.index, index=data['title']).drop_duplicates()


Function to clean text and remove stopwords

In [23]:
def clean(text):
    stop_words = set(stopwords.words('english'))
    text = str(text).lower()
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)


quick brown fox jumps lazy dog.


In [28]:
data['description'] = data['description'].apply(func=clean)

Using the Term-Frequency-Inverse Term frequency.

In [32]:
tf = TfidfVectorizer()
tf_mat = tf.fit_transform(data['description'])


In [34]:
tf_mat.shape

(8807, 19131)

In [38]:
print(tf_mat)

  (0, 6277)	0.16254396363689053
  (0, 11437)	0.30519046546477174
  (0, 5656)	0.2239952783710916
  (0, 9876)	0.12590836562639762
  (0, 6446)	0.22926768903690262
  (0, 9454)	0.35217431230742974
  (0, 9137)	0.3178095805166836
  (0, 16111)	0.27919454940575633
  (0, 4395)	0.18158280908846863
  (0, 8861)	0.3109717601819953
  (0, 3512)	0.3109717601819953
  (0, 18522)	0.22720271728772545
  (0, 7889)	0.16463147802014286
  (0, 6150)	0.20054299311773363
  (0, 8591)	0.33696767624101076
  (1, 4112)	0.3152131950220164
  (1, 12392)	0.2392888011846694
  (1, 12351)	0.23210859801850298
  (1, 2706)	0.33372291820649863
  (1, 17379)	0.18523436154323253
  (1, 16918)	0.18578261288918585
  (1, 15165)	0.20511050136745518
  (1, 13294)	0.23788214859625015
  (1, 18634)	0.2719475517695359
  (1, 13147)	0.2570889180420943
  :	:
  (8805, 6728)	0.1812611142527262
  (8805, 1305)	0.21508489436993197
  (8805, 14813)	0.3078660340693084
  (8805, 16533)	0.24299519154903884
  (8805, 3245)	0.2880564112126181
  (8805, 10858)	0

Calcuating the cosine simalirty matrix thus the movie's description would be matched with the description of all available descriptions mathematically

In [40]:
cos_sim = linear_kernel(tf_mat, tf_mat)

In [41]:
print(cos_sim[:10])

[[1.         0.         0.         ... 0.         0.0149629  0.        ]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.02230089]
 ...
 [0.         0.         0.         ... 0.         0.         0.03491323]
 [0.04016534 0.         0.         ... 0.         0.         0.        ]
 [0.01630652 0.         0.         ... 0.05836642 0.01539104 0.04158208]]


Autocorrect function using edit distance to be able to not to give error.

In [50]:
def autocorrect(title):
    closest_match = min(data['title'], key=lambda x: edit_distance.distance(title, x))
    return closest_match

Reccomendation function to autocorrect the user inputted movie name and then using it's description and with the help of cosine similarity index reccomend movies based on the most closest one.

In [60]:
def recc(title,num_recommend, cosine_sim=cos_sim):
    auto_titles = autocorrect(title)
    print('Do you mean : {} ?'.format(auto_titles))
    idx = indices[auto_titles]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_similar = sim_scores[1:num_recommend+1]
    movie_indices = [i[0] for i in top_similar]
    return data['title'].iloc[movie_indices]

driver

In [64]:
s = input("Enter movie name : ")
recc(s,10)

Enter movie name : inception 
Do you mean : Inception ?


,title
1939,Dedh Ishqiya
6304,"Big Dreams, Small Spaces"
4360,The World Is Yours
6726,Eyyvah Eyyvah 2
3796,Hyper HardBoiled Gourmet Report
1536,Incarnate
5202,Strange Weather
6148,American Warfighter
6620,Dog Eat Dog
1621,Quigley Down Under
